
### Python list comprehension

In [2]:
from gurobipy import *

In [6]:
# List comprehension
squares = [i**2 for i in range(6)]
squares

[0, 1, 4, 9, 16, 25]

In [10]:
# Generator expression
SumSquares = sum(i**2 for i in range(6))
SumSquares

55

In [12]:
pairs = [(i, j) for j in range(4) for i in range(j)]
pairs

[(0, 1), (0, 2), (1, 2), (0, 3), (1, 3), (2, 3)]

In [14]:
Nonsquares = [i for i in range(10) if i not in squares]
Nonsquares

[2, 3, 5, 6, 7, 8]

The Gurobi interface takes advantage of lists, list comprehension and generator expressions:

A Gurobi class to store lists of tuples

In [16]:
Cities = ['A', 'B', 'C', 'D']
Routes = tuplelist([('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D')])
Routes

<gurobi.tuplelist (5 tuples, 2 values each):
 ( A , B )
 ( A , C )
 ( B , C )
 ( B , D )
 ( C , D )
>

What makes it special: select statement for efficient filtering

In [9]:
for c in Cities:
    print(Routes.select(c, '*'))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( B , C )
 ( B , D )
>
<gurobi.tuplelist (1 tuples, 2 values each):
 ( C , D )
>
[]


The tuplelist is indexed to make select() efficient

### Indexed variables: Model.addVars

In [24]:
# Using integers
m = Model("model 1")
x = m.addVars(2, 3, name = 'x')
x

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>}

In [20]:
# Using lists of scalars
m = Model("model 2")
y = m.addVars(Cities, Cities, name = 'y')
y

{('A', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'A'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('D', 'D'): <gurobi.Var *Awaiting Model Update*>}

In [22]:
# Using a tuplelist
z = m.addVars(Routes, name = 'z')
z

{('A', 'B'): <gurobi.Var *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Var *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Var *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Var *Awaiting Model Update*>}

In [26]:
# Using a generator expression(!)
w = m.addVars((i for i in range(5) if i != 2), name = 'w')
w

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

### Indexed constraints: Model.addConstrs()

In [40]:
# Using a generator expression
m = Model("model")

x = m.addVars(Routes, name = "x")
y = m.addVars(Routes, name = "y")

m.addConstrs((x[i,j] + y[i,j] <= 2 for i,j in Routes), name = "capacity")

{('A', 'B'): <gurobi.Constr *Awaiting Model Update*>,
 ('A', 'C'): <gurobi.Constr *Awaiting Model Update*>,
 ('B', 'C'): <gurobi.Constr *Awaiting Model Update*>,
 ('B', 'D'): <gurobi.Constr *Awaiting Model Update*>,
 ('C', 'D'): <gurobi.Constr *Awaiting Model Update*>}

### Aggregate sum operator: Full

In [ ]:
# Using generator expression inside a quicksum function 

obj = quicksum(cost[i,j]*x[i,j] for i,j in arcs) 

# Remark: quicksum works just like Python's sum function, but it is more efficient for optimization models.  

### Aggregate sum: Simple

In [ ]:
# A tupledict of variables has a sum function, using the same syntax as tuplelist.select()

x = m.addVars(3, 4, vtype = GRB.BINARY, name = "x")
m.addConstrs(x.sum(i, "*") <= 1 for i in range(3))

In [44]:
# The above example generators the constraints:

x[0,0] + x[0, 1] + x[0, 2] + x[0, 3] <= 1
x[1,0] + x[1, 1] + x[1, 2] + x[1, 3] <= 1
x[2,0] + x[2, 1] + x[2, 2] + x[2, 3] <= 1

In [ ]:
# Dot product
# A tuple of variables has a prod() function to compute the dot product

# If cost is a dictionary, then the following are equivalent:

obj = quicksum(cost[i,j]*x[i,j] for i,j in arcs)
obj = x.prod(cost)